In [17]:
import pandas as pd 
import requests
import json
from datetime import datetime

In [18]:
JSON_URL = "https://wowtokenprices.com/history_prices_full.json"

In [19]:
def request_token_price(request):
    response = requests.get(request)
    response_json = response.json()
    return response_json['eu']

In [20]:
df = pd.DataFrame(request_token_price(JSON_URL))
df

,price,time
0,35000,1429646660
1,35350,1429647590
2,35704,1429648533
3,36061,1429649396
4,36422,1429650302
...,...,...
133371,164501,1610618603
133372,164330,1610619803
133373,164160,1610621003
133374,163919,1610622203


In [21]:
df['date']=df['time'].apply(lambda x: datetime.fromtimestamp(x))
df['day']=df['date'].apply(lambda x: x.weekday())
df['hour']=df['date'].apply(lambda x: x.hour)
df['date']=pd.to_datetime(df['date'])

In [22]:
df=df.set_index('date',drop=False)
df = df.sort_index()
df

,price,time,date,day,hour
date,,,,,
2015-04-21 22:04:20,35000,1429646660,2015-04-21 22:04:20,1,22
2015-04-21 22:19:50,35350,1429647590,2015-04-21 22:19:50,1,22
2015-04-21 22:35:33,35704,1429648533,2015-04-21 22:35:33,1,22
2015-04-21 22:49:56,36061,1429649396,2015-04-21 22:49:56,1,22
2015-04-21 23:05:02,36422,1429650302,2015-04-21 23:05:02,1,23
...,...,...,...,...,...
2021-01-14 11:03:23,164501,1610618603,2021-01-14 11:03:23,3,11
2021-01-14 11:23:23,164330,1610619803,2021-01-14 11:23:23,3,11
2021-01-14 11:43:23,164160,1610621003,2021-01-14 11:43:23,3,11


In [23]:
std = lambda x: (x.std())
df['7_day_std']=df['price'].rolling('7d').apply(std)
df['30_day_std']=df['price'].rolling('30d').apply(std)
df['7_day_mean']=df['price'].rolling('7d').mean()
df['30_day_mean']=df['price'].rolling('30d').mean()
df['1_day_mean']=df['price'].rolling('1d').mean()
df['7_day_low']=df['price'].rolling('7d').min()
df['30_day_low']=df['price'].rolling('30d').min()
df['7_day_high']=df['price'].rolling('7d').max()
df['30_day_high']=df['price'].rolling('30d').max()
df['1_day_high']=df['price'].rolling('1d').max()
df['1_day_low']=df['price'].rolling('1d').min()
df['last_change']=df['price'].diff(periods=2)
df=df.dropna()

In [24]:
df=df.rename(columns = {'price':'current_price'}, inplace = False)
df

,current_price,time,date,day,hour,7_day_std,30_day_std,7_day_mean,30_day_mean,1_day_mean,7_day_low,30_day_low,7_day_high,30_day_high,1_day_high,1_day_low,last_change
date,,,,,,,,,,,,,,,,,
2015-04-21 22:35:33,35704,1429648533,2015-04-21 22:35:33,1,22,352.001894,352.001894,35351.333333,35351.333333,35351.333333,35000.0,35000.0,35704.0,35704.0,35704.0,35000.0,704.0
2015-04-21 22:49:56,36061,1429649396,2015-04-21 22:49:56,1,22,456.629226,456.629226,35528.750000,35528.750000,35528.750000,35000.0,35000.0,36061.0,36061.0,36061.0,35000.0,711.0
2015-04-21 23:05:02,36422,1429650302,2015-04-21 23:05:02,1,23,562.104795,562.104795,35707.400000,35707.400000,35707.400000,35000.0,35000.0,36422.0,36422.0,36422.0,35000.0,718.0
2015-04-21 23:20:38,36786,1429651238,2015-04-21 23:20:38,1,23,668.330582,668.330582,35887.166667,35887.166667,35887.166667,35000.0,35000.0,36786.0,36786.0,36786.0,35000.0,725.0
2015-04-21 23:35:11,37154,1429652111,2015-04-21 23:35:11,1,23,775.556666,775.556666,36068.142857,36068.142857,36068.142857,35000.0,35000.0,37154.0,37154.0,37154.0,35000.0,732.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-14 11:03:23,164501,1610618603,2021-01-14 11:03:23,3,11,3166.878698,2900.562318,160615.431412,160477.311653,160190.458333,155955.0,155921.0,173013.0,173013.0,164501.0,156954.0,75.0
2021-01-14 11:23:23,164330,1610619803,2021-01-14 11:23:23,3,11,3169.944908,2901.541382,160618.821074,160478.149051,160224.277778,155955.0,155921.0,173013.0,173013.0,164501.0,156954.0,-137.0
2021-01-14 11:43:23,164160,1610621003,2021-01-14 11:43:23,3,11,3172.482302,2902.360209,160621.664016,160478.872629,160272.430556,155955.0,155921.0,173013.0,173013.0,164501.0,156954.0,-341.0


In [25]:
df.to_csv("Token-prices.csv",index=False)

In [35]:
df = pd.DataFrame(request_token_price(JSON_URL))
df['date']=df['time'].apply(lambda x: datetime.fromtimestamp(x))
df['day_of_week']=df['date'].apply(lambda x: x.weekday())
df['hour']=df['date'].apply(lambda x: x.hour)
df['date']=pd.to_datetime(df['date'])
df['month']=pd.to_datetime(df.date).dt.month
df.date=df.date.dt.floor('d')
df = df.sort_index()

In [41]:
price = df.groupby(by=['date']).mean().price


In [71]:
df2=pd.DataFrame()
df2['price']=price
df2['date']=df2.index
df2.reset_index(drop=True)
df2['month']=pd.to_datetime(df2.date).dt.month
df2['month']=pd.to_datetime(df2.date).dt.month
df2['day_of_week']=df2['date'].apply(lambda x: x.weekday())
df2.reset_index(drop=True)
df2.to_csv("Token-prices-avg-day.csv")

In [69]:
df2

,price,date,month,day_of_week
date,,,,
2015-04-21,36250.375000,2015-04-21,4,1
2015-04-22,41571.255319,2015-04-22,4,2
2015-04-23,34058.500000,2015-04-23,4,3
2015-04-24,37422.458333,2015-04-24,4,4
2015-04-25,34447.492537,2015-04-25,4,5
...,...,...,...,...
2021-01-10,160445.777778,2021-01-10,1,6
2021-01-11,162693.583333,2021-01-11,1,0
2021-01-12,163558.319444,2021-01-12,1,1
